# Función `cohend_group`
Implementación en un notebook para Google Colab.
Permite seleccionar las variables pre y post y el método para calcular el tamaño del efecto de Cohen.

In [ ]:
# Instalar dependencias en Colab (si es necesario)
!pip install -q xlsxwriter

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
def cohend_group(df: pd.DataFrame, varlist: list[str], group: str, method: str = 'dz') -> pd.DataFrame:
    """Calcula el tamaño del efecto de Cohen d por nivel de un grupo.
    Los valores 2 representan 'logro'. Se calcula la proporción de logros en las
    variables PRE y POST, su diferencia y el tamaño de efecto según el método
    indicado. También se exporta el resultado a un archivo Excel.
    """
    # 1. Validaciones iniciales
    if group not in df.columns:
        raise ValueError(f'La columna de grupo {group!r} no existe en el DataFrame')
    if len(varlist) % 2 != 0:
        raise ValueError("'varlist' debe contener un número par de variables")
    for col in varlist:
        if col not in df.columns:
            raise ValueError(f'La columna {col!r} no existe en el DataFrame')

    # 2. Separar en listas PRE y POST
    mid = len(varlist) // 2
    prevars = varlist[:mid]
    postvars = varlist[mid:]

    dfc = df.copy()
    # 3. Asegurar valores numéricos
    dfc[prevars + postvars] = dfc[prevars + postvars].apply(pd.to_numeric, errors='coerce')

    # 4. Proporciones y diferencia por fila
    dfc['tpre'] = (dfc[prevars] == 2).mean(axis=1)
    dfc['tpost'] = (dfc[postvars] == 2).mean(axis=1)
    dfc['dif'] = dfc['tpost'] - dfc['tpre']

    resultados = []
    # 5. Iterar por cada nivel de grupo
    for nivel in dfc[group].astype(str).unique():
        sub = dfc[dfc[group].astype(str) == str(nivel)]
        sub = sub.dropna(subset=['tpre', 'tpost'])
        N = len(sub)
        if N == 0:
            continue
        m_pre = sub['tpre'].mean()
        m_post = sub['tpost'].mean()
        m_diff = sub['dif'].mean()
        sd_pre = sub['tpre'].std(ddof=1)
        sd_post = sub['tpost'].std(ddof=1)
        sd_diff = sub['dif'].std(ddof=1)
        r_pp = sub['tpre'].corr(sub['tpost'])

        if method == 'dz':
            denom = sd_diff
        elif method == 'dav':
            denom = (sd_pre + sd_post) / 2
        elif method == 'drm':
            denom = np.sqrt(sd_pre**2 + sd_post**2 - 2 * r_pp * sd_pre * sd_post)
        else:
            raise ValueError("method debe ser 'dz', 'dav' o 'drm'")

        d_cohen = m_diff / denom if denom else np.nan

        if N >= 2:
            t_res = stats.ttest_rel(sub['tpost'], sub['tpre'], nan_policy='omit')
            p_value = t_res.pvalue
        else:
            p_value = np.nan

        if pd.isna(d_cohen):
            magnitude = 'NA'
            symbol = ''
        elif abs(d_cohen) >= 0.80:
            magnitude = 'GRANDE'
            symbol = '★★★'
        elif abs(d_cohen) >= 0.50:
            magnitude = 'MEDIANO'
            symbol = '★★'
        elif abs(d_cohen) >= 0.20:
            magnitude = 'PEQUEÑO'
            symbol = '★'
        else:
            magnitude = 'TRIVIAL'
            symbol = ''

        resultados.append({
            'grupo': nivel,
            'N': N,
            'm_pre': m_pre,
            'm_post': m_post,
            'm_diff': m_diff,
            'd_cohen': d_cohen,
            'p_value': p_value,
            'magnitude': magnitude,
            'symbol': symbol
        })

    result_df = pd.DataFrame(resultados)
    filename = 'cohend_resultados.xlsx'
    titulo = f"Cohen d – Variables: {', '.join(prevars)}, {', '.join(postvars)} (método={method})"
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        result_df.to_excel(writer, sheet_name='resultados', startrow=2, index=False)
        writer.sheets['resultados'].write('A1', titulo)
    print(f'Archivo guardado: {filename}')
    return result_df

## Seleccionar variables y método
Modifica las siguientes celdas para elegir las variables PRE y POST y el método para calcular d.

In [ ]:
# Listado de columnas disponibles en df
list(df.columns)

In [ ]:
# Editar manualmente la lista de variables (mitad PRE, mitad POST)
varlist = ['ce_p1', 'ce_p2', 'cs_i1_p1', 'cs_i1_p2']
# Método para el denominador: 'dz', 'dav' o 'drm'
method = 'dav'

In [ ]:
resultados = cohend_group(df, varlist, group='macro', method=method)
resultados